## Dissertation Early Modelling
### Gavin Rolls

Migrated from OvertureTest.ipynb. I've queried all of my Overture and OSM data in DataCollection.ipynb and done the initial cleaning in DataCleaning.ipynb. This document is dedicated running some early iterations of employment models

In [10]:
#Library Imports

#Basics
import pandas as pd
import geopandas as gpd

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

#Spatial Indexing
import h3

In [ ]:
#Import Cleaned Dataset

empl_geog = gpd.read_file('data/combined_data/empl_geog.geojson')

empl_geog.head()

## Basic Models

### Basic Model - Feature Counts (Overture) + Population Only

In [14]:
#Before incorporating any more data into my model, I want to see how well COUNTS alone can predict employment at LSOA level

#Identifying variables (feature variables vs target to predict)
features = ['num_overture_buildings', 'num_places', 'population']
target = 'total'

X = empl_geog[features]
y = empl_geog[target]

#Split into train/test - I know I need to make this spatially sensitive but we're just going with this for now
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Init/train model
model = LinearRegression()
model.fit(X_train, y_train)

#Predictions
y_pred = model.predict(X_test)

#Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 4460587.330033497
R^2 Score: 0.31047839937648836


### Basic Model - Feature Counts (OSM) + Population Only